# Final Project - file analyse Word 2 Vec

## Thanks to this code we will be able to define the 20 most common words use in a file text.
    1) The first step will be to clean the text (Natural Language Processing, lemmatize, elimate stopwords & punctuation, etc)
    2) Then, we create a graph to be use in Gephi
    
### We will realize this work with a nlp_sm (small base of words)

### Import database

In [1]:
import spacy
import re
import en_core_web_sm
import en_core_web_lg
import math
import token
import string

import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter

import gensim #W2V

# Pour générer un graphe Gephy
import networkx as nx

C:\Users\paulo\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
nlp_sm = en_core_web_sm.load()
nlp_sm.max_length = 1000000

In [3]:
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
punctuations = string.punctuation
numbers = string.digits

In [5]:
with open('Files/Anaerobic digestion for the stabilization of the organic fraction of municipal waste.txt') as fp:
    text = fp.read()

In [6]:
with open('Files/Anaerobic digestion of food waste - thesis.txt') as fp:
    text += fp.read()

In [7]:
with open('Files/What a Waste 2.0 A Global Snapshot of Solid Waste Management.txt') as fp:
    text += fp.read()

### Clean the text

In [8]:
# This function split a text into a list of sentences.
def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n\n\n\n\n\n","<stop>")
    text = text.replace("\n\n\n\n\n","<stop>")
    text = text.replace("\n\n\n\n","<stop>")
    text = text.replace("\n\n\n","<stop>")
    text = text.replace("\n\n","<stop>")
    text = text.replace("\n"," ")
    text = text.replace(". ",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

In [9]:
# This function split and clean each sentences of a list into a list of words.
def cleanup_sentences_sm(doc):
    words_list_sm = []
    for i in doc:
        sentence = nlp_sm(i)
        tokens = [tok.lemma_.lower().strip() for tok in sentence if tok.lemma_ != '-PRON-'] #lemmatize
        tokens = [tok for tok in tokens if tok not in stopwords] #delete stopwords
        tokens = [tok for tok in tokens if tok not in punctuations] #delete punctuation
        tokens = [tok for tok in tokens if not tok.isdigit()] #delete integers
        tokens = [tok for tok in tokens if len(tok) > 2] #delete letters alone
        tokens = [tok for tok in tokens if tok != 'et'] #delete the word 'et'
        tokens = [tok for tok in tokens if tok != 'al'] #delete the word 'al'
        tokens = [tok for tok in tokens if tok != 'angelidaki'] #delete the word 'angelidaki'
        tokens = [tok for tok in tokens if tok != 'kim'] #delete the word 'kim'
        tokens = [tok for tok in tokens if tok != 'epa'] #delete the word 'epa'
        tokens = [tok for tok in tokens if tok != 'ostrem'] #delete the word 'ostrem'
        tokens = [tok for tok in tokens if tok != 'dranco'] #delete the word 'dranco'
        tokens = [tok for tok in tokens if tok != 'dufferin'] #delete the word 'dufferin'
        tokens = [tok for tok in tokens if tok != 'garcia'] #delete the word 'garcia'
        tokens = [tok for tok in tokens if tok != 'del'] #delete the word 'del'
        tokens = [tok for tok in tokens if tok != 'kayhanian'] #delete the word 'dranco'
        tokens = [tok for tok in tokens if tok != 'vss'] #delete the word 'dufferin'
        tokens = ' '.join(tokens)
        word_list = [word.strip(string.punctuation) for word in tokens.split()] #create a list with each word

        #delete decimals and **/**
        to_delete = []
        for each in word_list:
            if re.search("^[0-9]*.[0-9]*$", each) is not None:
                to_delete.append(each)
            elif re.search("^[a-z0-9]*/[a-z0-9]*$", each) is not None:
                to_delete.append(each)
        for each in to_delete:
            word_list.remove(each)
        
        words_list_sm.append(word_list)

    return words_list_sm

In [10]:
# This function clean a text into a list of words.
def cleanup_text_sm(doc):
    doc = nlp_sm(doc)    
    tokens = [tok.lemma_.lower().strip() for tok in doc if tok.lemma_ != '-PRON-'] #lemmatize
    tokens = [tok for tok in tokens if tok not in stopwords] #delete stopwords
    tokens = [tok for tok in tokens if tok not in punctuations] #delete punctuation
    tokens = [tok for tok in tokens if not tok.isdigit()] #delete integers
    tokens = [tok for tok in tokens if len(tok) > 2] #delete letters alone
    tokens = [tok for tok in tokens if tok != 'et'] #delete the word 'et'
    tokens = [tok for tok in tokens if tok != 'al'] #delete the word 'al'
    tokens = [tok for tok in tokens if tok != 'angelidaki'] #delete the word 'angelidaki'
    tokens = [tok for tok in tokens if tok != 'kim'] #delete the word 'kim'
    tokens = [tok for tok in tokens if tok != 'epa'] #delete the word 'epa'
    tokens = [tok for tok in tokens if tok != 'ostrem'] #delete the word 'ostrem'
    tokens = [tok for tok in tokens if tok != 'dranco'] #delete the word 'dranco'
    tokens = [tok for tok in tokens if tok != 'dufferin'] #delete the word 'dufferin'
    tokens = [tok for tok in tokens if tok != 'garcia'] #delete the word 'garcia'
    tokens = [tok for tok in tokens if tok != 'del'] #delete the word 'del'
    tokens = [tok for tok in tokens if tok != 'kayhanian'] #delete the word 'dranco'
    tokens = [tok for tok in tokens if tok != 'vss'] #delete the word 'dufferin'
    tokens = ' '.join(tokens)
    words_list = [word.strip(string.punctuation) for word in tokens.split()] #create a list with each word

    #delete decimals
    to_delete = []
    for each in words_list:
        if re.search("^[0-9]*.[0-9]*$", each) is not None:
            to_delete.append(each)
        elif re.search("^[a-z0-9]*/[a-z0-9]*$", each) is not None:
            to_delete.append(each)
    for each in to_delete:
        words_list.remove(each)
            
    return words_list

In [11]:
# This function delete all the words which occure less than a number we define from a list of list.  
def most_common_words(list_W2V, doc, occurence):
    to_delete = []
    sentence_list = []
    counts = dict(Counter(doc))
    for word in counts.keys():
        if counts[word] <= occurence:
            to_delete.append(word)
    for sentence in list_W2V:
        word_list = []
        for word in sentence:
            if not word in to_delete:
                word_list.append(word)
        sentence_list.append(word_list)
    list_W2V = [each for each in sentence_list if len(each) > 0]
    
    return list_W2V

In [12]:
def create_graph(list_W2V):
    model_sm = gensim.models.Word2Vec(list_W2V, min_count = 5, size = 300)
    graph = nx.Graph()
    graph.add_nodes_from(list(model_sm.wv.vocab))
    for word in model_sm.wv.vocab:
        for s in model_sm.most_similar(word):
            graph.add_edge(word, s[0], weigh = s[1])
    print(nx.info(graph))
    return graph

In [13]:
sentences_sm = split_into_sentences(text)

In [15]:
W2V_sm = cleanup_sentences_sm(sentences_sm)

In [16]:
words_sm = cleanup_text_sm(text)

In [17]:
W2V_list_sm = most_common_words(W2V_sm, words_sm, 100)

### Creation of a graph file with only the words which occure more than 100 times

In [21]:
graph_occurences100 = create_graph(W2V_list_sm)
nx.write_gexf(graph_occurences100,'graphs/graph_occurences100_bis.gexf')

C:\Users\paulo\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


Name: 
Type: Graph
Number of nodes: 71
Number of edges: 537
Average degree:  15.1268


## Analyse graph_occurences100

The graph is available in the directory graphs in pdf or PNG. You can open the PNG by launching the next cell.

To do this graph, I used W2V, which define the similarity between words. I analyzed only the words which occure 100 or more times in the text.
Then on gephy, I used the spatialization "Force Atlas 2" to define the shape of the graph. Thanks to the statistical tool modularity, Gephy computed clusters and he found 4 groups of words. They are well separated in the graph.
The words which occure the most (according to the other study about Word_occurence) have a similarity less important than other, they appear in orange in the graph and they are really smaller than others.

In [20]:
from PIL import Image
myImage = Image.open("graphs/Graph_occurences100.PNG");
myImage.show();

### Creation of a graph file with all the words

In [22]:
graph_all = create_graph(W2V_sm)
nx.write_gexf(graph_all,'graphs/graph_all_bis.gexf')

C:\Users\paulo\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


Name: 
Type: Graph
Number of nodes: 1637
Number of edges: 16013
Average degree:  19.5638


## Analyse graph_all

The graph is available in the directory graphs in pdf or PNG. You can open the PNG by launching the next cell.

To do this graph, I used the same method that previously. This time, I analyzed all the words of the text, except the one that I cleaned. About the cleaning part, my code is really far away from being perfect. There is still some acronyms, references names and other stuff I would like to eliminate in order to make the text more clear.

Then on gephy, I used exactly the same method with "Force Atlas 2" spatialization and modularity to get clusters. Gephy found 5 groups of words.
The shape of the graph is quite elongated. There is three major clusters the blue, the light green and the dark green. At the top, there is the blue cluster, in the middle is the light green group and the dark green one is really at the bottom of the graph and represent the words which occure the most in the text, like waste, anaerobic, digestion etc. This position on the graph confirm that the similarity of these words are really low. They are used together and don't refer to many others.
The purple and orange clusters are less concentrated and seems to be linked with many other words.

In [23]:
from PIL import Image
myImage = Image.open("graphs/Graph_all.PNG");
myImage.show();